In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim import corpora, models, similarities

In [8]:
import csv
with open('Papers.csv', 'rb') as f:
    data = [row for row in csv.reader(f.read().splitlines())]

In [11]:
#Look at full data
limit = 3
l = 0
for d in data:
    print d
    l += 1
    if l > limit: break

['Id', 'Title', 'EventType', 'PdfName', 'Abstract', 'PaperText']
['5677', 'Double or Nothing: Multiplicative Incentive Mechanisms for Crowdsourcing', 'Poster', '5677-double-or-nothing-multiplicative-incentive-mechanisms-for-crowdsourcing.pdf', 'Crowdsourcing has gained immense popularity in machine learning applications for obtaining large amounts of labeled data. Crowdsourcing is cheap and fast, but suffers from the problem of low-quality data. To address this fundamental challenge in crowdsourcing, we propose a simple payment mechanism to incentivize workers to answer only the questions that they are sure of and skip the rest. We show that surprisingly, under a mild and natural no-free-lunch requirement, this mechanism is the one and only incentive-compatible payment mechanism possible. We also show that among all possible incentive-compatible  mechanisms (that may or may not satisfy no-free-lunch), our mechanism makes the smallest possible payment to spammers.  Interestingly, this u

In [14]:
#Look at one column of the data
limit = 3
l = 0
for d in data:
    print d[1] #title
    l += 1
    if l > limit: break

Title
Double or Nothing: Multiplicative Incentive Mechanisms for Crowdsourcing
Learning with Symmetric Label Noise: The Importance of Being Unhinged
Algorithmic Stability and Uniform Generalization


In [53]:
#LOAD ONE COLUMN OF THE DATA INTO A DOCUMENTS ARRAY
documents = []

limit = 100
l = 0
for d in data:
    documents.append(d[1]) #title
    l += 1
    if l > limit: break

documents = documents[1:]
print documents

['Double or Nothing: Multiplicative Incentive Mechanisms for Crowdsourcing', 'Learning with Symmetric Label Noise: The Importance of Being Unhinged', 'Algorithmic Stability and Uniform Generalization', 'Adaptive Low-Complexity Sequential Inference for Dirichlet Process Mixture Models', 'Covariance-Controlled Adaptive Langevin Thermostat for Large-Scale Bayesian Sampling', 'Robust Portfolio Optimization', 'Logarithmic Time Online Multiclass prediction', 'Planar Ultrametrics for Image Segmentation', 'Expressing an Image Stream with a Sequence of Natural Sentences', 'Parallel Correlation Clustering on Big Graphs', 'Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks', 'Space-Time Local Embeddings', 'A Convergent Gradient Descent Algorithm for Rank Minimization and Semidefinite Programming from Random Linear Measurements', 'Smooth Interactive Submodular Set Cover', 'Galileo: Perceiving Physical Object Properties by Integrating a Physics Engine with Deep Learning'

In [54]:
#REMOVE PUNCTUATION AND COMMON WORDS WITHOUT DISCRIMINATORY QUALITIES ("STOP WORDS")
import string
exclude = set(string.punctuation)
stoplist = set('for a of the and to in or on from using via with by as are'.split())
#texts = [[word for word in document.lower().split() if word not in stoplist] for document in documents]
texts = [[''.join(ch for ch in word if ch not in exclude) for word in document.lower().split() if word not in stoplist] for document in documents]
print texts

[['double', 'nothing', 'multiplicative', 'incentive', 'mechanisms', 'crowdsourcing'], ['learning', 'symmetric', 'label', 'noise', 'importance', 'being', 'unhinged'], ['algorithmic', 'stability', 'uniform', 'generalization'], ['adaptive', 'lowcomplexity', 'sequential', 'inference', 'dirichlet', 'process', 'mixture', 'models'], ['covariancecontrolled', 'adaptive', 'langevin', 'thermostat', 'largescale', 'bayesian', 'sampling'], ['robust', 'portfolio', 'optimization'], ['logarithmic', 'time', 'online', 'multiclass', 'prediction'], ['planar', 'ultrametrics', 'image', 'segmentation'], ['expressing', 'an', 'image', 'stream', 'sequence', 'natural', 'sentences'], ['parallel', 'correlation', 'clustering', 'big', 'graphs'], ['faster', 'rcnn', 'towards', 'realtime', 'object', 'detection', 'region', 'proposal', 'networks'], ['spacetime', 'local', 'embeddings'], ['convergent', 'gradient', 'descent', 'algorithm', 'rank', 'minimization', 'semidefinite', 'programming', 'random', 'linear', 'measurement

In [55]:
# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)

for text in texts:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 1] for text in texts]

from pprint import pprint
pprint(texts)

[[],
 ['learning'],
 ['algorithmic', 'generalization'],
 ['adaptive', 'inference', 'mixture', 'models'],
 ['adaptive', 'largescale', 'bayesian', 'sampling'],
 ['robust', 'optimization'],
 ['logarithmic', 'time', 'online', 'multiclass'],
 ['image'],
 ['image'],
 ['clustering'],
 ['object', 'detection', 'networks'],
 ['local', 'embeddings'],
 ['gradient', 'algorithm', 'rank', 'programming', 'random', 'linear'],
 ['smooth', 'submodular', 'cover'],
 ['object', 'deep', 'learning'],
 ['nearly', 'optimal'],
 ['neural', 'models'],
 ['bayesian', 'learning', 'linear', 'model'],
 ['learning'],
 ['fast', 'accurate', 'inference', 'models'],
 ['probabilistic', 'stochastic', 'optimization'],
 ['algorithmic', 'recurrent'],
 [],
 ['regret', 'bandits'],
 ['spectral', 'methods', 'gaussian', 'gaussian'],
 ['sample'],
 ['bidirectional', 'recurrent', 'convolutional', 'networks'],
 [],
 ['fast', 'algorithm', 'embeddings'],
 ['convolutional', 'neural', 'networks'],
 ['algorithms', 'distributed', 'estimation']

In [56]:
#ASSOCIATE EVERY WORD WITH AN INTEGER (FOR COMPUTATIONAL EFFICIENCY)
dictionary = corpora.Dictionary(texts)
pprint(dictionary.token2id)

{u'accurate': 37,
 u'adaptive': 4,
 u'algorithm': 24,
 u'algorithmic': 1,
 u'algorithms': 50,
 u'analysis': 74,
 u'approach': 59,
 u'arbitrary': 84,
 u'bandits': 43,
 u'bayesian': 7,
 u'bidirectional': 48,
 u'classification': 73,
 u'clustering': 17,
 u'compressive': 70,
 u'constrained': 66,
 u'convergence': 54,
 u'convolutional': 49,
 u'cover': 31,
 u'deep': 32,
 u'detection': 18,
 u'discrete': 69,
 u'distributed': 52,
 u'divergences': 64,
 u'dueling': 56,
 u'embeddings': 22,
 u'estimation': 51,
 u'estimators': 65,
 u'fast': 38,
 u'framework': 71,
 u'frankwolfe': 68,
 u'gaussian': 46,
 u'generalization': 2,
 u'gradient': 25,
 u'guarantees': 81,
 u'image': 16,
 u'inference': 6,
 u'kernel': 80,
 u'largescale': 8,
 u'learning': 0,
 u'linear': 23,
 u'local': 21,
 u'logarithmic': 14,
 u'machine': 72,
 u'methods': 45,
 u'mixture': 5,
 u'model': 36,
 u'models': 3,
 u'multiclass': 12,
 u'multilabel': 76,
 u'nearly': 34,
 u'network': 60,
 u'networks': 20,
 u'neural': 35,
 u'nonconvex': 61,
 u'o

In [57]:
#REPRESENT EVERY DOCUMENT AS A SPARSE ARRAY OF WORD IDS AND COUNTS (FOR COMPUTATIONAL EFFICIENCY)
corpus = [dictionary.doc2bow(text) for text in texts]
pprint(corpus)

[[],
 [(0, 1)],
 [(1, 1), (2, 1)],
 [(3, 1), (4, 1), (5, 1), (6, 1)],
 [(4, 1), (7, 1), (8, 1), (9, 1)],
 [(10, 1), (11, 1)],
 [(12, 1), (13, 1), (14, 1), (15, 1)],
 [(16, 1)],
 [(16, 1)],
 [(17, 1)],
 [(18, 1), (19, 1), (20, 1)],
 [(21, 1), (22, 1)],
 [(23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1)],
 [(29, 1), (30, 1), (31, 1)],
 [(0, 1), (19, 1), (32, 1)],
 [(33, 1), (34, 1)],
 [(3, 1), (35, 1)],
 [(0, 1), (7, 1), (23, 1), (36, 1)],
 [(0, 1)],
 [(3, 1), (6, 1), (37, 1), (38, 1)],
 [(11, 1), (39, 1), (40, 1)],
 [(1, 1), (41, 1)],
 [],
 [(42, 1), (43, 1)],
 [(44, 1), (45, 1), (46, 2)],
 [(47, 1)],
 [(20, 1), (41, 1), (48, 1), (49, 1)],
 [],
 [(22, 1), (24, 1), (38, 1)],
 [(20, 1), (35, 1), (49, 1)],
 [(50, 1), (51, 1), (52, 1)],
 [(6, 1), (7, 1), (52, 1), (53, 1)],
 [(0, 1)],
 [(6, 1), (23, 1), (29, 1), (54, 1), (55, 1)],
 [(43, 1), (56, 1)],
 [(18, 1), (33, 1), (57, 1)],
 [(12, 1)],
 [],
 [(58, 1)],
 [(0, 1), (15, 1), (39, 1)],
 [(0, 1), (6, 1)],
 [(7, 1), (9, 1), (35, 1), (59,

In [58]:
#COMPUTE AND APPLY MAPPING FROM COUNTS TO NUMBERS THAT ACCOUNT FOR DOCUMENT FREQUENCY
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)

[]
[(0, 1.0)]
[(1, 0.7071067811865475), (2, 0.7071067811865475)]
[(3, 0.36738718484669075), (4, 0.5594861350914039), (5, 0.6241798070319798), (6, 0.40299068170917685)]
[(4, 0.5290697347292825), (7, 0.45199632394506273), (8, 0.5290697347292825), (9, 0.4856642407333723)]
[(10, 0.7867204542473403), (11, 0.6173094255467501)]
[(12, 0.48684255211322786), (13, 0.5431364088823155), (14, 0.5431364088823155), (15, 0.4159206801118734)]
[(16, 1.0)]
[(16, 1.0)]
[(17, 1.0)]
[(18, 0.6231622539782471), (19, 0.6231622539782471), (20, 0.47258608785437306)]
[(21, 0.7446425758093053), (22, 0.6674634329250426)]
[(23, 0.3019249369090437), (24, 0.44416013578835944), (25, 0.44416013578835944), (26, 0.39812476298383426), (27, 0.44416013578835944), (28, 0.39812476298383426)]
[(29, 0.6193522206718214), (30, 0.5551589082185765), (31, 0.5551589082185765)]
[(0, 0.34887434607940315), (19, 0.6903938428225602), (32, 0.6337531320951139)]
[(33, 0.7071067811865475), (34, 0.7071067811865475)]
[(3, 0.6545884203064519), (35

In [59]:
#PRODUCE LSI MODELS FOR TOPIC NUMBERS FROM 2 TO 10
for N in range(2,11):
    print N, 'topics:'
    lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=N) 
    corpus_lsi = lsi[corpus_tfidf]
    pprint(lsi.print_topics(N))
    print

2 topics:
[(0,
  u'-0.863*"learning" + -0.221*"deep" + -0.173*"online" + -0.153*"models" + -0.150*"inference" + -0.136*"neural" + -0.115*"networks" + -0.101*"optimization" + -0.088*"bayesian" + -0.080*"object"'),
 (1,
  u'-0.503*"neural" + -0.486*"models" + -0.415*"networks" + 0.305*"learning" + -0.290*"convolutional" + -0.204*"recurrent" + -0.165*"bidirectional" + -0.144*"analysis" + -0.108*"mixture" + -0.095*"optimization"')]

3 topics:
[(0,
  u'0.863*"learning" + 0.221*"deep" + 0.173*"online" + 0.153*"models" + 0.150*"inference" + 0.136*"neural" + 0.115*"networks" + 0.101*"optimization" + 0.088*"bayesian" + 0.080*"object"'),
 (1,
  u'0.503*"neural" + 0.486*"models" + 0.415*"networks" + -0.305*"learning" + 0.290*"convolutional" + 0.204*"recurrent" + 0.165*"bidirectional" + 0.144*"analysis" + 0.108*"mixture" + 0.095*"optimization"'),
 (2,
  u'-0.701*"optimization" + 0.338*"models" + -0.260*"online" + 0.229*"inference" + -0.215*"robust" + -0.207*"nonconvex" + -0.139*"stochastic" + -0.1

In [60]:
#PRODUCE LDA MODELS FOR TOPIC NUMBERS FROM 2 TO 10
for N in range(2,11):
    print N, 'topics:'
    lda = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=N,passes=10,alpha='auto')
    pprint(lda.print_topics(N)) #ugly print-out
    print

2 topics:
[(0,
  u'0.066*learning + 0.057*models + 0.048*inference + 0.042*linear + 0.031*bayesian + 0.025*sampling + 0.020*neural + 0.020*network + 0.020*random + 0.019*analysis'),
 (1,
  u'0.043*networks + 0.041*optimization + 0.036*learning + 0.030*online + 0.026*neural + 0.025*recurrent + 0.025*sparse + 0.025*object + 0.024*bandits + 0.024*model')]

3 topics:
[(0,
  u'0.149*learning + 0.037*object + 0.037*deep + 0.029*model + 0.029*stochastic + 0.029*detection + 0.020*structure + 0.020*cover + 0.020*bayesian + 0.020*online'),
 (1,
  u'0.090*models + 0.073*inference + 0.038*sparse + 0.029*linear + 0.029*sample + 0.029*multiclass + 0.022*neural + 0.021*classification + 0.020*recurrent + 0.020*convergence'),
 (2,
  u'0.051*optimization + 0.042*neural + 0.036*sampling + 0.036*bandits + 0.036*robust + 0.032*networks + 0.028*rank + 0.028*nonconvex + 0.028*analysis + 0.027*convolutional')]

4 topics:
[(0,
  u'0.145*learning + 0.047*online + 0.036*inference + 0.036*bandits + 0.036*sample +

In [61]:
#SECOND RUN, NICER PRINT OUT, INCLUDES TOPIC COHERENCE SCORES
for N in range(2,11):
    print N, 'topics:'
    lda = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=N,passes=10,alpha='auto')
    pprint(lda.top_topics(corpus, num_words=10))
    print

2 topics:
[([(0.076473571869567436, u'learning'),
   (0.048444056764272034, u'optimization'),
   (0.042736182230298937, u'networks'),
   (0.041708434030939184, u'inference'),
   (0.03304607950299801, u'neural'),
   (0.030722557231334432, u'linear'),
   (0.025719424782210263, u'object'),
   (0.02568072188754628, u'recurrent'),
   (0.025598859941865254, u'deep'),
   (0.024593100114612805, u'model')],
  -59.235215135921877),
 ([(0.053805737767143272, u'models'),
   (0.026763377425102798, u'learning'),
   (0.025186132906226857, u'sampling'),
   (0.025111581215807234, u'bandits'),
   (0.02454560548421534, u'sparse'),
   (0.022337011722352109, u'online'),
   (0.019679917498222864, u'embeddings'),
   (0.019596496887625627, u'multiclass'),
   (0.019518370882269216, u'fast'),
   (0.01944721633358952, u'spectral')],
  -59.834803258743229)]

3 topics:
[([(0.042293580816170503, u'bayesian'),
   (0.042192731943763211, u'inference'),
   (0.034380442948330585, u'model'),
   (0.033671512432640202, u's

In [62]:
#THIRD RUN, INCLUDES TOPIC COHERENCE SCORES AND TOPICS PER DOCUMENT
for N in range(2,11):
    print N, 'topics:'
    lda = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=N,passes=10,alpha='auto')
    pprint(lda.top_topics(corpus, num_words=10))
    print 'topics per document when topic number is set to',N
    for doc in corpus:
        pprint(lda.get_document_topics(doc))
    print
    print

2 topics:
[([(0.049998129016298735, u'optimization'),
   (0.044640000763435966, u'linear'),
   (0.042700776906663546, u'networks'),
   (0.029341527858251597, u'convolutional'),
   (0.027791873855029323, u'sparse'),
   (0.027659059052982365, u'recurrent'),
   (0.026861107241755812, u'object'),
   (0.025793207121116032, u'robust'),
   (0.022170716979106875, u'random'),
   (0.02211108342800262, u'classification')],
  -54.543902024796076),
 ([(0.088633999676740211, u'learning'),
   (0.051695903189847535, u'models'),
   (0.04214763393529903, u'inference'),
   (0.032965261384186574, u'neural'),
   (0.027945535905131477, u'online'),
   (0.026749188015213268, u'bayesian'),
   (0.022941342159089256, u'bandits'),
   (0.022775016821593063, u'sampling'),
   (0.019359245088609141, u'deep'),
   (0.017861068824737227, u'largescale')],
  -62.721906294758497)]
topics per document when topic number is set to 2
[(0, 0.56340026468796589), (1, 0.43659973531203405)]
[(0, 0.71962746992278659), (1, 0.28037253

In [66]:
#FOURTH RUN, INCLUDES TOPIC COHERENCE SCORES (AND SUMMARY STATS ABOUT THEM) AND TOPICS PER DOCUMENT

import numpy
for N in range(2,11):
    print N, 'topics:'
    lda = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=N,passes=10,alpha='auto')
    pprint(lda.top_topics(corpus, num_words=10))
    coherence_scores = []
    for topic in lda.top_topics(corpus, num_words=10):
        print 'coherence:', topic[1]
        coherence_scores.append(topic[1])
    print 'min coherence', min(coherence_scores)
    print 'max coherence', max(coherence_scores)
    
    scores = numpy.array(coherence_scores)
    print 'mean score',numpy.mean(scores)
    print 'std score',numpy.std(scores)
    
    print 'topics per document when topic number is set to',N
    for doc in corpus:
        pprint(lda.get_document_topics(doc))
    print
    print

2 topics:
[([(0.060768649962027389, u'models'),
   (0.047837250464587734, u'inference'),
   (0.02960173445472571, u'linear'),
   (0.026354846373044749, u'sparse'),
   (0.025586142850594511, u'robust'),
   (0.024681986675232324, u'bandits'),
   (0.02298539800919961, u'sampling'),
   (0.020536191286773173, u'embeddings'),
   (0.02045978125125392, u'adaptive'),
   (0.020376553264310433, u'analysis')],
  -57.822028860682039),
 ([(0.094806522510097815, u'learning'),
   (0.041160259539736545, u'optimization'),
   (0.036788727971721731, u'networks'),
   (0.029842818307833315, u'neural'),
   (0.02749960129862928, u'online'),
   (0.024662888313133388, u'object'),
   (0.024625790169833857, u'deep'),
   (0.024379034444208632, u'model'),
   (0.022378181127796239, u'recurrent'),
   (0.019123252084061817, u'spectral')],
  -57.845406955502192)]
coherence: -57.8220288607
coherence: -57.8454069555
min coherence -57.8454069555
max coherence -57.8220288607
mean score -57.8337179081
std score 0.0116890474

In [72]:
for i,doc in enumerate(documents[0:11]):
    print i,doc

0 Double or Nothing: Multiplicative Incentive Mechanisms for Crowdsourcing
1 Learning with Symmetric Label Noise: The Importance of Being Unhinged
2 Algorithmic Stability and Uniform Generalization
3 Adaptive Low-Complexity Sequential Inference for Dirichlet Process Mixture Models
4 Covariance-Controlled Adaptive Langevin Thermostat for Large-Scale Bayesian Sampling
5 Robust Portfolio Optimization
6 Logarithmic Time Online Multiclass prediction
7 Planar Ultrametrics for Image Segmentation
8 Expressing an Image Stream with a Sequence of Natural Sentences
9 Parallel Correlation Clustering on Big Graphs
10 Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks
